In [ ]:
import os #여기부터 실행하세요. one drive에서 파일 다운받아두세요. 
import re
import openpyxl
import calendar
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import time

cal = calendar.Calendar()

query = "여성"
query2 = "#2_" + query
year = 2019
xlist = list()

for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            dd = str(year) + str(month).zfill(2) + str(day).zfill(2)
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#2_html2text', query2, '#2_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#2_"+dd+"_"+query))
            xlist.append(xlsx_name)

In [ ]:

def replace_sub(text):
                
        from nltk.tokenize import regexp_tokenize
        from string import punctuation

        tlist, ttlist= list(), list()
        text = regexp_tokenize(text, r'[^\s\.][^\.\n]+')
        for _ in text:
            if len(_) > 11:
                tlist.append(_)
                
        text = ".\n".join(tlist) + "."

        text = re.sub(r'\d+\.(\n)\d+', r'', text)
        text = re.sub(r'\b(\w|[.])+@(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'\bhttps?://\w+(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-하-ㅣ\.?!\n ]', ' ', text)
        text = re.sub(r'\b[a-z][A-Za-z0-9]+\b', ' ', text)
        text = re.sub(r'\s{2,}', ' ', text) 
        text = re.sub(r'[\(\[].*?[\)\]]', '', text)
        text = re.sub('【.*】','',text)
        text = re.sub(r'"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\/:;<=>\@\\[\\\\\\]\\^_`\\{\\|\\}\\~', '', text)
        # 여기부터 새로운 태그 추가
        #신문사 이름은 나중에 한꺼번에 뺄 것이니 추가하지 말 것
        text = re.sub(r"-\d+-", '', text) # 이거 전에 '-'제거하면 안됨.
        text = re.sub(r"\b\w{3}\s기자", '', text)
        text = re.sub(r'\s?[a-z]\s', ' ', text)
        text = re.sub(r'\s+\d+px?', '', text)
        text = re.sub(r'\s\d+\s\d+\s\d+\s', '', text)
        text = re.sub(r'\s?(\d+\s)', '', text)
        text = re.sub(r'((\d+[a-z])+)', '', text)
        text = re.sub(r'[A-Z0-9a-z][A-Z0-9a-z]', '', text)
        text = re.sub(r'\b[A-Z0-9]\b', '', text)
        
        text = re.sub('해당 언론사가.*?가시겠습니까?','',text,flags=re.DOTALL)
        text = re.sub('무단전재.*?분류했습니다.','',text,flags=re.DOTALL)
        text = re.sub('기사 섹션 분류 안내.*?따르고 있습니다.','',text,flags=re.DOTALL)
        text = re.sub('네이버에서.*?클릭','',text,flags=re.DOTALL)
        text = re.sub('제한된 글자수 자를.*?비밀번호 가 일치 하지 않습니다.','',text,flags=re.DOTALL)
        text = re.sub('네이버에서.*?채널 구독하기','',text,flags=re.DOTALL)
        
        text = text.replace('출처', '').replace('방송캡처', '').replace('-', '').replace('오류를 우회하기 위한 함수 추가.', '')
        text = text.replace('동영상 뉴스.', '').replace('아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.', '')
        text = text.replace(' 사진 ', ' ').replace('경남신문의 콘텐츠는 저작권법의 보호를 받는 바 무단전재 복사 재배포를 금합니다.', '')
        text = text.replace('돋움', '').replace('메인이미지', '').replace('. p .', "\n").replace('정상처리되지 않았습니다.', '')
        text = text.replace(' p ', '\n').replace('IMGFLOATING ', '').replace('캡쳐 ', '').replace(' P ', '').replace('INSERT RANDOM NUMBER HERE', '')
        text = text.replace('OptimaA Scrip Ta', '').replace('시원하게 털어드립니다.', '').replace('저작권자 인사이트 무단전재 및 재배포 금지 .', '')
        text = text.replace('수직배너 와이드', '').replace('www.', '').replace('동영상이 안 나오면', '').replace('여기를 누르시면 동영상을 별도의 새 창에서 볼 수 있습니다.', '')
        
        text = text.replace('취재기자 심충만 방송일조회', '').replace('게시글을 r로 보내기', '').replace('게시글을 으로 보내기', '').replace('게시글을 로 보내기', '')
        text = text.replace('이 글을 페이스북으로 퍼가기', '').replace('이 글을 트위터로 퍼가기', '').replace('이 글을 카카오스토리로 퍼가기', '').replace('이 글을 밴드로 퍼가기', '')
        text = text.replace('요즘 게시글을 요즘으로 보내기', '').replace('E에 있는 정보를 기본으로 가져오며', '').replace('해당 테이블에 등록된 정보가 없을 시', '')
        text = text.replace('모바일에 등록된 4를 가져온다', '').replace('롱리브더킹 임시 해결', '').replace('오른쪽 마우스 부정클릭 방지', '')
        text = text.replace('카툰 유료화 안내 표시', '').replace('기사본문 광고 제거', '').replace('! !','').replace('? ?', '')
        text = text.replace('기자의 다른 기사 보기','').replace('뉴광고 기사 배너지면 가변','').replace('기사 모바일 우측상단','')
        text = text.replace('게시물을 삭제하시겠습니까?','').replace('로그인하신 후에 이용해 주시기 바랍니다!','').replace('본문상단우측','').replace('공식 이미지','')
        text = text.replace('기자가 쓴 다른기사 보기','').replace('해당 언론사가 주요기사로 직접 선정한 기사입니다','').replace('언론사 편집판 바로가기','')
        text = text.replace('게티이미지뱅크','').replace('함수 를 구현합니다','').replace('언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다','')
        text = text.replace('헉! 소리나는 스!토리 뉴스 헉스','').replace('클릭해 뉴스 들어간다 뉴스쿨','').replace('이 기사를 추천합니다','')
        text = text.replace('본문 텍스트A','').replace('포토 위젯 시작 문의','').replace('포토 위젯 종료 문의','').replace('게티이미지','')
        text = text.replace('이전데이터에서 사진이 추가될 경우 본문 삽입처리','').replace('기사본문 오른쪽 정렬','')
        text = text.replace('본문내 상단우측','').replace('아티클 공통 기자','').replace('아티클 공통 공유 및 댓글','')
        text = text.replace('새창 페이스북으로 이동 페이스북','').replace('새창 트위터로 이동 트위터','').replace('새창 카카오스토리로 이동 카카오스토리','')
        text = text.replace('댓글로 이동 댓글 수','').replace('아티클 공통 본문','').replace('아티클 공통 부제','')
        text = text.replace('기사 페이지 본문내우측','').replace('이미지 확대보기','').replace('에스엔에스 공유경제신문','').replace('에스엔에스에스비에스화면','') 
        
        text = text.replace('저작권자 입법국정전문지 더리더 e 무단전재 및 재배포 금지 .','').replace('머니투데이 더리더','')
        text = text.replace('사진','').replace('에너지경제 온에어','').replace('뉴스','')
        text = text.replace('제공 안양시 안양 파이낸셜','').replace('매직스트로베리사운드 제공.','')
        text = text.replace('정하은기자','').replace('캡처 헤럴드P','').replace('경기도청 전경 군포시민신문.','').replace('독자가 내는 소중한 월 1천원 구독료는 군포시민신문 대부분의 재원이자 올바른 지역언론을 지킬 수 있는 힘입니다.','')
        text = text.replace('충북 제천경철서 홈페이지 캡처.','').replace('페이스북 캡처','').replace('김민성 인턴기자','').replace('S썸바이벌캡처','')
        text = text.replace('저작권자 스타트업투데이 무단전재 및 재배포 금지.','').replace('로그인 후 이용 가능합니다.','').replace('인스타그램 캡처','')
        text = text.replace('이 에디터의 다른 글 보기.','').replace('저작권자 아트인사이트 .','').replace('추천 비추천은 회원만 가능합니다.','').replace('값이 제대로 넘어오지 않았습니다.','').replace('등록된 코멘트가 없거나 코멘트 글이 아닙니다.','')
        text = text.replace('자신의 글에는 추천 또는 비추천 하실 수 없습니다.\n이미 추천 하신 글 입니다.\n이미 비추천 하신 글 입니다.','').replace(' 페이스북 페이지 캡처','')
        text = text.replace('디지털 콘텐츠 이용규칙 보기','').replace('조혜진기자 앵커','').replace('권은주기자','').replace('! 뉴데일리 뉴데일리','').replace('출품 여성작가들과 임종국 교수 오른쪽','')
        text = text.replace('출품 작가 김미숙 김종태 신보경 신소윤 안선문 인상희 임순옥 임윤상 한경진 홍은희 홍태돈 지도교수 임종국 .','')
        text = text.replace('전시회는 7월8일까지 서울교대 샘미술관 아카데미 실용반 활동영상 .','').replace('E평생교육원실용반그룹전 g영상','').replace('A아래에 작가별 대표작들을 소개한다','')
        text = text.replace('무단전재 재배포 금지','').replace('보도자료 및 기사제보','').replace('자유민주 시장경제의 파수꾼 뉴데일리 ','').replace('기사','')
        text = text.replace('인터플레이 중앙중단','')
        text = text.replace('라이브리 댓글 작성을 위해 를 활성화 해주세요','').replace('뉴데일리 댓글 운영정책','')
        text = text.replace('왼쪽부터 임호천씨박주식씨김해정씨.','').replace('이상훈 우철훈 김정근 선임기자','').replace('제공 e','')
        text = text.replace('우창범 트위터 웹데일리','').replace('오애희 웹데일리 기자','')
        text = text.replace('저작권자 웹데일리 무단 전재 및 재배포 금지 .','').replace('저작권자 기호일보 무단전재 및 재배포 금지','').replace('정상처리되지 않았습니다!','')
        text = text.replace('원하는곳에 붙여넣기','').replace('하루 김명민 이종석과 다정하게 한 컷','').replace(' 모바일 우측상단','').replace('김유진 공유경제신문 기자','').replace('공유경제가 제공하는 콘텐츠에 대해 독자는 친근하게 접근할 권리와 정정ㆍ반론ㆍ추후 보도를 청구 할 권리가 있습니다','').replace('저작권자 공유경제신문 무단 전재 및 재배포 금지','')
        text = text.replace('이건희 공유경제신문 기자','').replace('기부와 문화인류학 기부의 성립조건','').replace('공유 시선 공동체 자치 가 공유지의 비극 해결','')
        text = text.replace('공유 시선 서구 문학에서 장애는 어떻게 재현되나.\n공경기획 기업의 공유가치창출 기업가치를 높이는 경영전략.','').replace('산책 선순환 방식의 기부문화 나와야','')
        text = text.replace('기부와 문화인류학 기부와 경제.','').replace('공경기획 공유경제 플랫폼의 다양화로 협력적 소비 이끌어','').replace('산책 프로보노 사회의 변화를 추구하는 기부 방식','')
        text = text.replace('공경기획 공유경제 서비스 영역의 확대','').replace('공경기획 숙박 공유 플랫폼 지역사회 경제 기여','').replace('공경기획 공유경제는 소유에서 시작된다','')
        text = text.replace('저작권자 대구 경북 대표지역언론 대구일보 ','').replace('폴리 박현 기자','').replace('월 일모바일한경 에 게재된 입니다','').replace('모바일한경 더보기 ','').replace('모바일한경은 프리미엄 채널입니다','').replace('한국정책방송원 무단전재 및 재배포 금지','').replace('대한 9회 클립영상','').replace('영상취재 김명신 홍성주 영상편집 양세형','')
        text = text.replace('저작권자 로이슈 무단 전재 및 재배포 금지','').replace('제보 및 보도자료 제공','').replace('에디터에서 들어간 가 아닐 경우 태그 씌우지 않음','').replace('이 는 언론사에서 사회 섹션으로 분류했습니다','').replace('네이버에서 뉴시스 채널 구독하기 뉴시스 빅데이터 주가시세표 바로가기 저작권자 공감언론 뉴시스통신사','').replace('의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다','').replace('이 는 사용자 추천 으로 모바일 메인 판에 노출된 이력이 있습니다','')
        text = text.replace('무단전재 및 재배포 금지','').replace('저작권자 헤드라인제주','')
        text = text.replace('메인페이지가 로드됩니다','').replace('본문 중하단 광고 서인선','').replace('의견글을 삭제하시겠습니까?','').replace('제공 의정부시 의정부 스포츠서울','').replace('후원 금액을 선택해주세요','').replace('정기후원 페이지로 이동합니다','')
        text = text.replace('팝업 차단를 해제하고 이용하시기 바랍니다','').replace('팝업창을 활성화 해주세요','').replace('가 마음에 드셨나요? ','').replace('학교비정규직노조 울산지부 제공','').replace('문체부 게시물 분석와 무관한 이미지입니다','').replace('이미지 로이터연합','').replace('류원혜 인턴기자','').replace('김고금평 기자','').replace('타다 문제 계약해지 성 인지 교육 강화 저작권자','').replace('무단복제재배포 금지 네이버 홈에서 C 채널 구독하기 엠빅분토론 심상정양경규 진보에 길을 묻다 살충제 이겨낸 바퀴벌레의 초특급 진화','').replace('이 는 모바일 메인 판 에 노출된 이력이 있습니다','')
        text = text.replace('제한된 글자수 자를 초과하였습니다','').replace('닉네임 을 입력해 주세요','').replace('비밀번호 를 입력해 주세요','').replace('건전한 댓글문화 정착을 위해 이용에 주의를 부탁드립니다','').replace('명예훼손 개인정보 유출 인격권 침해 허위사실 유포 등은 이용약관 및 관련법률에 의해 제재를 받을 수 있습니다.','').replace('댓글내용 을 입력해 주세요','').replace('양수미 공유경제신문 기자','').replace('양수미 공유경제신문 기자','').replace('무단 전재 및 재배포 금지','').replace('꿈을 담는 캔버스 채널A ','').replace('저작권 언론사 하이닥 무단 전재 및 재배포 금지','').replace('저작권자 패션엔미디어','').replace('저작권자 중앙일보플러스 무단전재 및 재배포금지 ','').replace('채널A 무료 보기 네이버에서 채널A 구독하기 보기 판결의 재구성 모자 실종 반전의 잔혹극 신고자가 살해 용의자','')
        text = text.replace('김수지기자','').replace('무단 전재재배포 금지','').replace('공보닷컴 트위터 캡처 헤럴드경제 정지은 인턴기자','').replace('이미지 연합 아시아경제 조슬기나 기자','')
        text = text.replace('제공 삼성화재 스포츠서울 삼성화재는 어','').replace(' 열매 아프리카 방송 캡처온라인','').replace('저작권 한국일보','').replace('이지웅 웹데일리 기자','')
        text = text.replace('글 이혁재 소아시한의원 원장 주간경향','').replace('글 이혁재 소아시한의원 원장 주간경향','').replace('의 디지털이용규칙에 따른 저작권을 행사합니다','')
        text = text.replace('제공 아메바컬쳐아메바컬쳐','').replace('위 의 법적인 책임과 권한은 에듀동아에 있습니다','').replace('에듀동아 전수완 인턴 기자','').replace('인라이플 본문 삽입','').replace('저작권자 무단복제재배포 금지','').replace('등록된 댓글이 없습니다','')
        text = text.replace('로그인 후 댓글 등록이 가능합니다','').replace('지금 로그인 하시겠습니까','').replace('주소를 클립보드에 복사하시겠습니까?','').replace('이 글의 L주소입니다.','').replace('를 눌러 클립보드로 복사하세요','').replace('비즈 인터플레이 중앙중단','')
        text = text.replace('보도자료제목','').replace('제공 아메바컬쳐힙합그룹','').replace('스포츠서울 김자영기자','').replace('로이슈가 제공하는 콘텐츠에 대해 독자는 친근하게 접근할 권리와 정정ㆍ반론ㆍ추후 보도를 청구 할 권리가 있습니다.','')     
        
        text = text.replace('C뉴스 제희원입니다.','').replace('취재기자 제희원 방송일조회.','').replace('뉴스돋보기 여성장애인으로 산다는 것 .','')
        text = text.replace('연합뉴스 김동욱입니다.','').replace('방송화면 캡처','').replace('방송 영상 캡처','').replace('윤소윤 인턴기자','').replace('최진실기자','')
        text = text.replace('지원하지 않는 브라우저 입니다.','').replace('백호튜터링','').replace('트위터 제공','').replace('?제공','')
        text = text.replace('트위터 보내기 트위터.','').replace('페이스북 보내기 페이스북.','').replace('네이트온 보내기 네이트온 보내기.','')
        text = text.replace('제작 이태호 연합','').replace('영상취재 김태형 영상편집 양세형 .','')
        text = text.replace('인사이트','').replace('배너지면','').replace('라이브리 프리미엄 설치코드','').replace('선거전 기존 코드','')
        text = text.replace('면 경제섹션','').replace('?p 뉴시스 웹데일리 .','').replace('신세계푸드 광주기아챔피언스필드 신세계푸드 제공','').replace('류빈 기자','')
        
        text = text.replace('스크랩하기','').replace('배너클릭','').replace('상단 배너 광고','').replace('내용 모바일 배너','').replace('하단 배너 광고','')
        text = text.replace('다른 사람의 권리를 침해하거나 명예를 훼손하는 댓글은 관련 법률에 의해 제제를 받을 수 있습니다.','')
        text = text.replace('타인에게 불쾌감을 주는 욕설 등 비하하는 단어들은 표시가 제한됨을 알려드립니다.','')
        text = text.replace('유튜브 계정 캡처','').replace('로그인 후 작성이 가능합니다','').replace('C광주방송 C를 검색해서 보실 수 있습니다.','')
        text = text.replace('방송 캡처','').replace('넷플릭스 제공','').replace('페이스북 .','').replace('트위터 .','').replace('구글플러스 .','')
        text = text.replace('?루갈 박선호 입대 전 막지막 화보 복근 드러낸 멋진 슈트룩','').replace('?김보라 블랙 티셔츠에 직접 T까지! 우주인 조안 화제 만발 .','').replace('?뉴발란스 김연아 허훈 김하성 오승환 참여 질주본능 캠페인 .','').replace('?지컷 빨라진 신상품 출시 예년 보다 빠른 간절기 컬렉션 전개 .','')
        text = text.replace('?제니퍼 알렉스 화제의 연상연하! 한여름 밤의 쿨한 데이트 커플룩 .','').replace('?엑셀시오르 어드벤처러의 정신과 영감 담은 하이커 출시 .','').replace('?안젤리나 졸리 막내 아들과 쇼핑 나들이! 하이라이트는 X마스크 .','').replace('?한예슬 이집트 여왕! 압도적인 존재감의 팜므파탈 뷰티룩 .','')
        text = text.replace('?레깅스의 반란! 멋스럽게 연출하는 일상 레깅스룩 .','').replace('?패션엔 포토 트롯 여신 송가인 기분좋은 퇴근길! 시선 끄는 퍼프 슬리브! .','').replace('?서예지 송윤아 둘다 우아해! 같은 후프 귀걸이 어디꺼? .','').replace('?멜빵 바지의 귀환! 다시 뜨는 추억의 오버롤 패션 .','')
        text = text.replace('?패션 업계 벌써 겨울! 다운 패딩 선판매에 나선 이유? .','').replace('?세계 2위 섬유의류 수출국 베트남의 위기! 세계 패션시장 위협하나? .','').replace('?푸마 강다니엘와 협업 스니커즈 컬렉션 전개 .','').replace('?모두 내가 결정해 스타일리스트 없이 옷 잘입는 스타 셀럽들 .','')
        text = text.replace('?케이트 미들턴 이유있는 패션 규칙! 컬러 드레스 향연.','').replace('?이효리 공효진 현아 벙거지 모자 벗킷햇 으로 끝내는 핫섬머룩 .','').replace('?메건 마클 왕실 떠나도 완판녀! 만원짜리 린넨 원피스 주문 폭주 .','').replace('?미스터트롯 임영웅 아웃도어 브랜드 밀레 모델 발탁 .','')
        text = text.replace('?티셔츠 하나면 끝! 지프 레트로 티셔츠 컬렉션 인기폭발 .','').replace('?패션엔 포토 가수 제시 눈누난나! 출근길의 도발적인 레깅스 핏! .','').replace('?지지 하디드 임신 7개월째.','').replace('아름다운 D라인 깜짝 공개 .','')
        text = text.replace('?케이트 미들턴 애정 뚝뚝 자녀 사랑! 3남매를 위한 엄마표 생축샷 .','').replace('?카일리 제너 딸사랑 과했나? 2살짜리 딸 명품 가방 구설수 .','').replace('?셀레나 고메즈 못말리는 빨간 맛 사랑! 레드룩 .','').replace('?그 옷 어디꺼 선미 깜짝 등장! 화사한 플라워 원피스 어디꺼? .','')
        text = text.replace('?밀레 임영웅 효과 볼까? 임영웅 빅로고 재킷 예약 판매 .','').replace('?메건 마클 여름에 더 끌려! 매력적인 스트라이프 스타일.','').replace('?이효리 군살 제로! 요가로 다져진 애슬레저 핏 .','').replace('아이디룩할머니 카디건이 유행? 대신 짧게 입고 하나만 입어야 인싸! .','')
        text = text.replace('쉬즈미스한혜진 빨라진 여름 패션고민 끝! 도시 여성의 우아한 섬머룩 .','').replace('?푸마 강다니엘과 협업 스니커즈 컬렉션 전개 .','')
        text = text.replace('아티클 공통 필진의 최신 글 보기','').replace('아티클 공통 주요','').replace('아티클 공통 추천','').replace('아티클 공통 공유','').replace('아티클 공통 태그','').replace('새창 구글플러스로 이동 구글','').replace('새창 핀터레스트로 이동 핀터레스트','').replace('새창 구글플러스로 이동 구글','').replace('새창 카카오톡으로 이동 카카오톡','')
        text = text.replace('새창 네이버블로그로 이동 네이버블로그','').replace('게시판 관리기준에 의거하여 작성해 주시기 바랍니다.','').replace('네이버 메인에서 구독하기','')
        text = text.replace('스크랩은 회원에게만 지원되는 기능입니다 .','').replace('이미 스크랩한 입니다.','').replace('로그인 후 이용 하실 수 있습니다.','').replace('홈페이지 캡처','')
        text = text.replace('메일보내기 기능은 회원에게만 제공되는 기능 입니다.','').replace('오늘을 진단하고 내일의 청을 제시하는 C 서브타이틀','').replace('앵커멘트','').replace('데스크 앵커','')
        text = text.replace('네이버에서 채널 구독하기','').replace('유튜브에서 N스타 채널 구독하고 선물 받아 가세요','').replace('모두에게 보여주고 싶은 라면','').replace('ㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴㄴ','')
        text = text.replace('위 은 내용과 직접적인 관련이 없음.','').replace('이미지 제공 삼성물산패션 빅데이터','').replace('누르면 을 더 크게 볼 수 있습니다','')
        text = text.replace('에프엔엔터테인먼트 제공','').replace('o e','').replace('a c','').replace('편집자 주 본고는 자료 제공사에서 제공한 것으로 연합는 내용에 대해 어떠한 편집도 하지 않았음을 밝혀 드립니다.','')
        text = text.replace('페이스북 페이스북 공유하기','').replace('트위터 트위터 공유하기','').replace('라인 라인 공유하기','').replace('네이버블로그 네이버블로그 공유하기','').replace('네이버밴드 네이버밴드 공유하기','')
        text = text.replace('네이버 네이버 공유하기','').replace('페이스북 화면 캡처','')
        text = text.replace('? 박용수 다른 보기 .','').replace('?6년 만에 룸 하차하는 손석희 후임은 .','').replace('?장병완 홍준표 인지능력 공감 부족.','').replace('?기름값 올라 3주 만의 상승 .','')
        text = text.replace('?신한은행 채용비리 오늘 밤 구속 결정 .','').replace('재정지원으로 해결될까? .','').replace('?단독 해썹 가공장 곰팡이와 오염물질 매년 적발 .','').replace('?단독 식약처 해썹업체관리 부실 축산물 위생 비상 .','')
        text = text.replace('?단독 식약처 무허가 가공장 단속 적발돼도 해썹은 유지 .','').replace('?한국식품안전관리인증원 해썹관리 매년 허술하게 점검해 .','').replace('?축산물 가공장 금속검출기 고장 쇳조각 걸러내지 못해 .','').replace('?김부겸 거침없이 가겠다 .','')
        text = text.replace('?소비자연대 쿠팡에 방역비용 청구 검토해야 주장 .','').replace('?김도읍 추미애 장관 윤석열 검찰총장 권한 무력화 해 .','').replace('?홍준표 좌파들 뻔뻔 년전 대출받아 산 집은 한채다 .','').replace('?종합 이인영 어떤 경우에도 남북 북미간 대화 끊이지 않아야 .','')
        text = text.replace('주호영 여당 일하는 국회법 은 야당 무력화법 독재 고속도로 .','').replace('?단독 쿠팡의 거짓말 거짓을 진실로 포장 .','').replace('?윤건영 박지원 국정원장 내정 탕평인사의 끝판 .','')
        text = text.replace('?해썹 가공 작업장 곳곳에 곰팡이.','').replace('구멍 숭숭 해썹 심사 .','').replace('?단독 해썹 가공장 곰팡이와 오염물질 매년 적발 .','').replace('? 이메일무단수집거부 .','').replace('한국탐사보도저널리즘 .','')
        text = text.replace('경기도 성남시 분당구 서현로 0번길 경기 아.','').replace('발행인 편집인 박동경 .','').replace('청소년보호책임자 김 준 .','').replace('개인정보관리 책임자 이현기 .','').replace('개인정보관리 책임자 이현기 .','')
        text = text.replace('후플러스의 모든 콘텐츠 영상 는 저작권법의 보호를 받은바 무단 전재와 복사 배포 등을 금합니다.','').replace('? 류지원 다른 보기 .','').replace('?김태년 한명숙 사법농단 피해자 법무부가 진실 밝혀야 .','').replace('?육아휴직 8명 중 1명은 아빠 대기업 쏠림 은 여전 .','')
        text = text.replace('?의협복지부 충돌 예고 .','').replace('?음악 스트리밍 창작자 몫 인상 추진 음원 이용료 오를까 .','').replace('?서울 북부지법 화장실 내 민원인 개인정보 노출.','')
        text = text.replace('픽스 버즈 F블루투스 이어폰 .','').replace('픽스 밸런스 인바디 체중계 .','').replace('픽스 쿨 휴대용 선풍기 .','').replace('픽스 바디체크 인바디 체중계 .','')
        text = text.replace('좋아요 를 누르시면 제일 빠른 소식을 전달해 드립니다.','').replace('이 광고문제로 로딩이 안되는 오류 발생','').replace('상단의 배너를 누르시면 바이오센추리 r 원문을 보실 수 있습니다.','').replace('방송 화면 캡처','')
        text = text.replace('날개광고 여백광고 .','').replace('하단 메인 반복 모듈 .','')
        text = text.replace('모든 브라우저 신규 플레시플레이어 없음 .','').replace('구 익스버전 기존 플레시플레이어 있는듯.','').replace('당신의 제보가 가 됩니다 N은 여러분의 소중한 제보를 기다립니다.','')
        text = text.replace('페이스북 공유하기','').replace('구글플러스 구글플러스 공유하기','').replace('싸이월드 싸이월드 공유하기','').replace('마이피플 공유하기','').replace('네이트온쪽지 네이트온쪽지 공유하기','').replace('구글북마크 구글북마크 공유하기','')
        text = text.replace('다른 곳에 퍼가실 때는 아래 고유 링크 주소를 로 사용해주세요.','').replace('클립보드에 복사하였습니다.','').replace('n원하시는 곳에 v키로 붙여 넣으세요.','')
        text = text.replace('기자프로필 노출여부 설정','').replace('프로필은 하단에 위의 과 함께 제공됩니다.','').replace('기자프로필 노출여부 설정 e','')
        text = text.replace('아이프레임 레이어팝업을 위한 설정 k는','').replace('p의 바로 아래 저장','').replace('레이어팝업시 배경마스킹을 위한 레이어','')
        text = text.replace('아이프레임을 부모창 정 가운데 레이어팝업으로 띄우는 경우 자식창에 ','').replace('p를 바로 인클루드 해도 무방','').replace('화면의 높이와 너비를 구한다.','')
        text = text.replace('마스크의 높이와 너비를 화면 것으로 만들어 전체 화면을 채운다.','').replace('애니메이션 효과 일단 1초동안 까맣게 됐다가불투명도로 간다.','').replace('로그인 후 의견을 등록하시면 자신의 의견을 관리하실 수 있습니다.','')
        text = text.replace('내용 저작권 등 다른 사람의 권리를 침해하거나 명예를 훼손하는 게시물은 이용약관 및 관련법률에 의해 제재를 받으실 수 있습니다.','')
        text = text.replace('나눔바른고딕 i e','')
        
        try: 
            text = text[:text.rindex('다.')+2]
            text = text.split(sep='.')
            for _ in text:
                if len(re.sub(r'\s{2,}', ' ', _)) > 11:
                    ttlist.append(re.sub(r'\s{2,}', ' ', _))
                    
            text = ".".join(ttlist) + "."
            return text
        except:
            return 0

In [ ]:
"""
유미 0-91 >>>>>>>>>> 0,100
지은 92-182 >>>>>>>>>>
은서 183-273 >>>>>>>>>>
유리 274-364 >>>>>>>>>>
"""

df = pd.read_excel(xlist[0]) # 여기에 숫자 바꿔가면서 하면 돼

n = 100
text = df.iloc[n, 3]
url = df.iloc[n, 4]
print(url)
print(replace_sub(text))
#re.findall(r'((\d+[a-z])+)', text)

In [ ]:
import requests
from bs4 import BeautifulSoup

tag = "div.cont_cont"

raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
html = BeautifulSoup(raw.text, "html.parser")
html.select(tag)